In [1]:
import json
import pandas as pd
import plotly.express as px
from pathlib import Path

DATA_DIR = Path("../..") / "data"
PEERS_PATH = DATA_DIR / "peers" / "peers.json"
ONLINE_PATH = DATA_DIR / "online_peers" / "online_peers.json"

PEERS_PATH, ONLINE_PATH

(PosixPath('../../data/peers/peers.json'),
 PosixPath('../../data/online_peers/online_peers.json'))

In [2]:
def load_json(path):
    with open(path, "r") as f:
        return json.load(f)

peers_raw = load_json(PEERS_PATH)
online_raw = load_json(ONLINE_PATH)

print("Total peers scanned:", len(peers_raw))
print("Total online peers:", len(online_raw))

Total peers scanned: 3828
Total online peers: 1665


In [3]:
df_peers = pd.DataFrame(peers_raw)
df_online = pd.DataFrame(online_raw)

df_peers.head()

,host,ssl,tcp,raw
0,137.184.244.174,NaN,NaN,"[137.184.244.174, 137.184.244.174, [v1.4.3, s5..."
1,194.233.69.180,NaN,50001.0,"[194.233.69.180, electrumx.dev, [v1.4.3, s5000..."
2,37.27.18.174,NaN,NaN,"[37.27.18.174, 37.27.18.174, [v1.4.3, s50002]]"
3,18.221.79.132,NaN,50001.0,"[18.221.79.132, btc5.publicrypto.com, [v1.4.3,..."
4,18.217.9.91,50002.0,50001.0,"[18.217.9.91, btc.byte-share.com, [v1.4.3, s60..."


In [4]:
df_peers = df_peers.drop_duplicates(subset="host")
df_online = df_online.drop_duplicates(subset="host")

In [5]:
df = df_peers.merge(df_online, on="host", how="left")
df["is_online"] = df["latency_ms"].notna()

In [6]:
total = len(df)
reachable = df["is_online"].sum()
unreachable = total - reachable

metrics = pd.DataFrame({
    "category": ["Reachable", "Unreachable"],
    "count": [reachable, unreachable]
})

metrics

,category,count
0,Reachable,100
1,Unreachable,191


In [7]:
fig = px.bar(
    metrics,
    x="category",
    y="count",
    text="count",
    title="Electrum Network Reachability Distribution",
    color="category",
    color_discrete_sequence=["#4CAF50", "#F44336"]
)

fig.update_layout(
    template="plotly_dark",
    height=450,
    xaxis_title="Status",
    yaxis_title="Number of Servers",
)
fig.update_traces(textposition="outside")

fig.show()

In [8]:
fig_pie = px.pie(
    metrics,
    values="count",
    names="category",
    title="Reachability Ratio",
    color="category",
    color_discrete_sequence=["#4CAF50", "#F44336"]
)

fig_pie.update_layout(template="plotly_dark", height=450)
fig_pie.show()

In [9]:
proto_stats = pd.DataFrame({
    "protocol": ["SSL only", "TCP only", "Both missing"],
    "count": [
        df["ssl"].notna().sum(),
        df["tcp"].notna().sum(),
        df[ (df["ssl"].isna()) & (df["tcp"].isna()) ].shape[0]
    ]
})

proto_stats

,protocol,count
0,SSL only,101
1,TCP only,141
2,Both missing,137


In [12]:
fig_proto = px.bar(
    proto_stats,
    x="protocol",
    y="count",
    text="count",
    title="Electrum Server Protocol Availability",
    color="protocol",
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig_proto.update_layout(
    template="plotly_dark",
    height=450,
    xaxis_title="Protocol",
    yaxis_title="Servers",
)
fig_proto.update_traces(textposition="outside")

fig_proto.show()

In [13]:
fig_proto.write_image(str(EXPORT_DIR / "protocol_availability.png"), scale=2)
print("Exported to site/public/results/protocol_availability.png")

Exported to site/public/results/protocol_availability.png


In [11]:
EXPORT_DIR = Path("../../site/public/results")
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

fig.write_image(str(EXPORT_DIR / "reachability_distribution.png"), scale=2)
print("Exported to site/public/results/reachability_distribution.png")

Exported to site/public/results/reachability_distribution.png
